In [4]:
from haystack.components.embedders import OpenAIDocumentEmbedder, OpenAITextEmbedder
from haystack_integrations.document_stores.weaviate.document_store import WeaviateDocumentStore
from haystack_integrations.components.retrievers.weaviate.embedding_retriever import WeaviateEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator

embedder = OpenAIDocumentEmbedder(model="text-embedding-3-small")
text_embedder = OpenAITextEmbedder(model="text-embedding-3-small")
document_store = WeaviateDocumentStore(url="http://localhost:8088")
retriever = WeaviateEmbeddingRetriever(document_store=document_store, top_k=3)

template = """
Answer the question only using the following context. Do not use any external information. 
If the answer is not present in the context, please answer with "I don't know".

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
prompt_builder = PromptBuilder(template=template)

generator = OpenAIGenerator(model="gpt-4o-mini")


/home/kartikeya/miniconda3/envs/argonk/lib/python3.12/site-packages/haystack/components/generators/openai.py:106: DeprecationWarning: In the upcoming releases 'gpt-3.5-turbo' will be replaced by 'gpt-4o-mini' as the default model
  warnings.warn(


## Manually embed query and fetch context

Embed

In [5]:
from haystack import Document

question = "What caused dinosaurs to become extinct?"
document = Document(content=question)

result = embedder.run(documents=[document])

embedded_document = result["documents"][0]
embedding = embedded_document.embedding

print("Embedding:", embedding)


Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.01it/s]

Embedding: [0.01488286629319191, 0.028380408883094788, 0.010614573024213314, -0.006187153980135918, -0.03472668677568436, 0.008742514066398144, 0.010464807972311974, 0.008550628088414669, -0.0033439649268984795, 0.02207157202064991, 0.025722086429595947, -0.011363396421074867, -0.042420849204063416, -0.028099600225687027, -0.01224326342344284, -0.030383512377738953, 0.024523967877030373, 0.04919770359992981, -0.07413352280855179, -0.0014204245526343584, -0.012561514042317867, -4.488187641982222e-06, 0.004790130071341991, 0.017204219475388527, 0.010127837769687176, 0.023737704381346703, 0.01965661533176899, -0.009514737874269485, -0.024261880666017532, -0.03588736429810524, 0.012458550743758678, -0.02209029160439968, 0.00806389283388853, 0.030121423304080963, -0.018533380702137947, -0.013394580222666264, 0.00925264973193407, 0.044330351054668427, 0.0042893546633422375, -0.03373449668288231, 0.0490104965865612, -0.04942234978079796, -0.003327584359794855, 0.036055851727724075, -0.0205552

Retrieve

In [6]:
retrieval_result = retriever.run(query_embedding=embedding)

print(retrieval_result)

retrieved_documents = retrieval_result["documents"]

for doc in retrieved_documents:
    print(doc.content)

{'documents': [Document(id=f547552c-85a1-44e7-a0ab-1195cf30b70f, content: 'All non-avian dinosaurs and most lineages of birds became extinct in a mass extinction event, called...', meta: {'split_id': 112.0, 'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'file_path': 'dinosaur-page.html'}, score: 0.8353862166404724, embedding: vector of size 1536), Document(id=bda7c905-e2f1-48b5-a2c2-8804c3082bf5, content: 'When dinosaurs appeared, they were not the dominant terrestrial animals. The terrestrial habitats we...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 42.0, 'file_path': 'dinosaur-page.html'}, score: 0.8047996759414673, embedding: vector of size 1536), Document(id=47f9c1b0-233a-4f8f-9c3f-12fa3e45c0c1, content: 'The Cretaceous–Paleogene extinction event, which occurred approximately 66 million years ago at the ...', meta: {'split_id': 47.0, 'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621

In [7]:
retrieved_documents

[Document(id=f547552c-85a1-44e7-a0ab-1195cf30b70f, content: 'All non-avian dinosaurs and most lineages of birds became extinct in a mass extinction event, called...', meta: {'split_id': 112.0, 'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'file_path': 'dinosaur-page.html'}, score: 0.8353862166404724, embedding: vector of size 1536),
 Document(id=bda7c905-e2f1-48b5-a2c2-8804c3082bf5, content: 'When dinosaurs appeared, they were not the dominant terrestrial animals. The terrestrial habitats we...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 42.0, 'file_path': 'dinosaur-page.html'}, score: 0.8047996759414673, embedding: vector of size 1536),
 Document(id=47f9c1b0-233a-4f8f-9c3f-12fa3e45c0c1, content: 'The Cretaceous–Paleogene extinction event, which occurred approximately 66 million years ago at the ...', meta: {'split_id': 47.0, 'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5

> **Warning:** There is an issue with embeddings. Two chunks with the same content exist. Investigate! (fixed later)

In [8]:
print(document_store.count_documents())

135


> Embeddings were added twice, once in notebook 5_embed_and_store.ipynb and later in 6_indexing_pipeline.ipynb

Generate

In [9]:
context = "\n".join([doc.content for doc in retrieved_documents])

prompt = f"Question: {question}\nContext: {context}\nAnswer:"

generation_result = generator.run(prompt=prompt)

generated_replies = generation_result["replies"]
for reply in generated_replies:
    print("Generated Reply:", reply)

Generated Reply: Dinosaurs became extinct as a result of the Cretaceous–Paleogene (K-Pg) extinction event, which occurred approximately 66 million years ago. This mass extinction was primarily caused by two major factors: an extraterrestrial impact event, such as the one that created the Chicxulub crater in the Yucatán Peninsula, and extensive flood basalt volcanism in India. The combination of these catastrophic events likely led to severe environmental changes that disrupted ecosystems globally, contributing to the extinction of about 76% of all species on Earth, including non-avian dinosaurs. Factors such as the relatively large size of most dinosaurs and their low diversity of smaller species, along with specific habitat dependencies of some avian lineages, may have also played a role in their extinction.


In [10]:
from haystack import Pipeline

query_pipeline = Pipeline()

query_pipeline.add_component("text_embedder", text_embedder)
query_pipeline.add_component("retriever", retriever)
query_pipeline.add_component("prompt_builder", prompt_builder)
query_pipeline.add_component("llm", generator)

query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
query_pipeline.connect("retriever", "prompt_builder.documents")
query_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - text_embedder: OpenAITextEmbedder
  - retriever: WeaviateEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [11]:
question = "Why are dinosaurs so fascinating?"

response = query_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])


/home/kartikeya/miniconda3/envs/argonk/lib/python3.12/site-packages/posthog/client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
/home/kartikeya/miniconda3/envs/argonk/lib/python3.12/site-packages/posthog/request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


Dinosaurs are fascinating because of their fantastic appearance and often enormous size, which have captured the popular imagination and made them an enduring part of human culture.


### Experiments

**Exp-1**

The word 'Gymnosperm' occurs only once in the 'Dinosaur' wiki page - it occurs in hte 3rd or 4th sentence of a passage chunk of about 10 sentences. 

Test if the embeddings are able to capture the word in the semantic meaning.

In [12]:
question = "How are gymnosperm plants related to dinosaurs?"

response = query_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

In the Late Triassic, gymnosperm plants, particularly conifers, radiated and served as a potential food source for dinosaurs.


Not bad. Satisfactory answer!

**Exp-2**



In [13]:
question = "What is Proceratosauridae?"

response = query_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

Proceratosauridae are tyrannosauroids with head crests.


Amazing! The word 'Proceratosauridae' occurs deep inside a very long chunk - was not expecting a pure vector embedding based retrieval to do the job. Suspected that a hybrid strategy (vector + BM25) may be required. 

**Exp-3**

In [14]:
question = "What is Averostra?"

response = query_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

Averostra is characterized as "bird snouts."


Ok the answer makes sense. But how did the LLM know that Averostra refers to a clade of theropod dinosaurs? The only reference to Averostra is the following - "Averostra ("bird snouts")". 

Modified the prompt to emphasize LLM not to use external information and to reply with "I don't know" if answer not in context.

Run manually to check which chunks are fetched.

In [15]:
from haystack import Document

question = "What is Averostra?"

document = Document(content=question)
result = embedder.run(documents=[document])
embedded_document = result["documents"][0]
embedding = embedded_document.embedding

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.96it/s]


In [16]:
retrieval_result = retriever.run(query_embedding=embedding)
print(retrieval_result)
retrieved_documents = retrieval_result["documents"]
for doc in retrieved_documents:
    print(doc.content)

{'documents': [Document(id=d060a7dc-17f6-43e6-9b3a-4365480a5543, content: '- †Ornithischia ("bird-hipped"; diverse bipedal and quadrupedal herbivores) - †Saphornithischia ("tr...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 57.0, 'file_path': 'dinosaur-page.html'}, score: 0.6415892839431763, embedding: vector of size 1536), Document(id=294a6ad7-9781-4bfb-9de4-5d1dc44d9c85, content: '- †Sauropodomorpha (herbivores with small heads, long necks, and long tails) - †Unaysauridae (primit...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 59.0, 'file_path': 'dinosaur-page.html'}, score: 0.6411151885986328, embedding: vector of size 1536), Document(id=0617d027-ae05-4321-921c-44b5dd2a18f3, content: '- †Alvarezsauroidea (small hunters with reduced forelimbs) - †Alvarezsauridae (insectivores with onl...', meta: {'split_id': 60.0, 'source_id': 'cccb7d931975c65babedb9ad3867015939fba6806219

The chunk which has the answer is ranked 2 out of 3. Also the score of the top ranked chunk ~ 0.64, so not a very confident retrieval(?). What if we could give only one chunk as context?

In [17]:
retrieval_result = retriever.run(query_embedding=embedding, top_k=1)
print(retrieval_result)
retrieved_documents = retrieval_result["documents"]
for doc in retrieved_documents:
    print(doc.content)

{'documents': [Document(id=d060a7dc-17f6-43e6-9b3a-4365480a5543, content: '- †Ornithischia ("bird-hipped"; diverse bipedal and quadrupedal herbivores) - †Saphornithischia ("tr...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 57.0, 'file_path': 'dinosaur-page.html'}, score: 0.6415892839431763, embedding: vector of size 1536)]}
- †Ornithischia ("bird-hipped"; diverse bipedal and quadrupedal herbivores) - †Saphornithischia ("true" ornithischians) - †Heterodontosauridae (small herbivores/omnivores with prominent canine-like teeth) - †Genasauria ("cheeked lizards") - †Thyreophora (armored dinosaurs; bipeds and quadrupeds) - †Eurypoda (heavy, quadrupedal thyreophorans) - †Stegosauria (spikes and plates as primary armor) - †Huayangosauridae (small stegosaurs with flank osteoderms and tail clubs) - †Stegosauridae (large stegosaurs) - †Ankylosauria (scutes as primary armor) - †Parankylosauria (small, southern ankylosaurs with macuahuitl-li

The answer does not exist in the top ranked chunk. What does the LLM model say now?

In [18]:

context = "\n".join([doc.content for doc in retrieved_documents])

prompt = (
    f"Answer the question only using the following context. "
    f"Do not use any external information. If the answer is not present in the context, "
    f"please answer with \"I don't know\".\n\n"
    f"Question: {question}\n"
    f"Context:\n{context}\n"
    f"Answer:"
)
generation_result = generator.run(prompt=prompt)

generated_replies = generation_result["replies"]
for reply in generated_replies:
    print("Generated Reply:", reply)

Generated Reply: I don't know.


As expected, the LLM responds with "I don't know" as the answer does not exist in the context of the only chunk that was retrieved.

This is a good opportunity to test BM25 based retrieval.

In [19]:
from haystack_integrations.components.retrievers.weaviate.bm25_retriever import WeaviateBM25Retriever


bm25retriever = WeaviateBM25Retriever(document_store=document_store)

In [20]:
bm25_retrieval_result = bm25retriever.run(query=question, top_k=1)
print(bm25_retrieval_result)
bm25_retrieved_documents = bm25_retrieval_result["documents"]
for doc in bm25_retrieved_documents:
    print(doc.content)

{'documents': [Document(id=a0ec45c3-c0cd-4604-a176-594456f11b36, content: 'Scholarly descriptions of what would now be recognized as dinosaur bones first appeared in the late ...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 23.0, 'file_path': 'dinosaur-page.html'}, score: 1.9453471899032593, embedding: vector of size 1536)]}
Scholarly descriptions of what would now be recognized as dinosaur bones first appeared in the late 17th century in England. Part of a bone, now known to have been the femur of a Megalosaurus , was recovered from a limestone quarry at Cornwell near Chipping Norton, Oxfordshire, in 1676. The fragment was sent to Robert Plot, Professor of Chemistry at the University of Oxford and first curator of the Ashmolean Museum, who published a description in his The Natural History of Oxford-shire (1677). He correctly identified the bone as the lower extremity of the femur of a large animal, and recognized that it was to

In [21]:
question

'What is Averostra?'

Unsatisfactory! Expected BM25 based embedding to return the proper chunk which had the term 'Averostra'.

In [22]:
bm25_retrieval_result = bm25retriever.run(query="What is MRCA", top_k=1)
print(bm25_retrieval_result)
bm25_retrieved_documents = bm25_retrieval_result["documents"]
for doc in bm25_retrieved_documents:
    print(doc.content)

{'documents': [Document(id=7ea8178b-0ad5-4b37-80bf-aa925decf8bb, content: 'Under phylogenetic nomenclature, dinosaurs are usually defined as the group consisting of the most r...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 5.0, 'file_path': 'dinosaur-page.html'}, score: 2.5518620014190674, embedding: vector of size 1536)]}
Under phylogenetic nomenclature, dinosaurs are usually defined as the group consisting of the most recent common ancestor (MRCA) of Triceratops and modern birds (Neornithes), and all its descendants. It has also been suggested that Dinosauria be defined with respect to the MRCA of Megalosaurus and Iguanodon , because these were two of the three genera cited by Richard Owen when he recognized the Dinosauria. Both definitions cover the same known genera: Dinosauria = Ornithischia + Saurischia. This includes major groups such as ankylosaurians (armored herbivorous quadrupeds), stegosaurians (plated herbivorous qu

But BM25 is able to find the term 'MRCA' as expected. So mixed results!

Next, let's check if 'MRCA' is found in single chunk vector embedding retrieval.

In [23]:
from haystack import Document

question = "What is MRCA"

document = Document(content=question)
result = embedder.run(documents=[document])
embedded_document = result["documents"][0]
embedding = embedded_document.embedding

retrieval_result = retriever.run(query_embedding=embedding, top_k=1)
print(retrieval_result)
retrieved_documents = retrieval_result["documents"]
for doc in retrieved_documents:
    print(doc.content)

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.94it/s]

{'documents': [Document(id=7ea8178b-0ad5-4b37-80bf-aa925decf8bb, content: 'Under phylogenetic nomenclature, dinosaurs are usually defined as the group consisting of the most r...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 5.0, 'file_path': 'dinosaur-page.html'}, score: 0.6315962672233582, embedding: vector of size 1536)]}
Under phylogenetic nomenclature, dinosaurs are usually defined as the group consisting of the most recent common ancestor (MRCA) of Triceratops and modern birds (Neornithes), and all its descendants. It has also been suggested that Dinosauria be defined with respect to the MRCA of Megalosaurus and Iguanodon , because these were two of the three genera cited by Richard Owen when he recognized the Dinosauria. Both definitions cover the same known genera: Dinosauria = Ornithischia + Saurischia. This includes major groups such as ankylosaurians (armored herbivorous quadrupeds), stegosaurians (plated herbivorous qu

Weaviate vector embedding is also able to find the term MRCA in the right chunk.


Results are mixed
- Both vector embedding retrieval and BM25 were **unable** to find the correct chunk for 'What is Averostra?'
- Both vector embedding retrieval and BM25 were **able** to find the correct chunk for 'What is MRCA'

**Exp-4** - Run more tests to check if BM25 retriever would add value

Q - What is Mamenchisaurus?

The term 'Mamenchisaurus' appears only once in the text in a chunk which is reasonably long.

BM25

In [24]:
bm25_retrieval_result = bm25retriever.run(query="What is Mamenchisaurus?", top_k=1)
print(bm25_retrieval_result)
bm25_retrieved_documents = bm25_retrieval_result["documents"]
for doc in bm25_retrieved_documents:
    print(doc.content)

{'documents': [Document(id=a0ec45c3-c0cd-4604-a176-594456f11b36, content: 'Scholarly descriptions of what would now be recognized as dinosaur bones first appeared in the late ...', meta: {'split_id': 23.0, 'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'file_path': 'dinosaur-page.html'}, score: 1.9453471899032593, embedding: vector of size 1536)]}
Scholarly descriptions of what would now be recognized as dinosaur bones first appeared in the late 17th century in England. Part of a bone, now known to have been the femur of a Megalosaurus , was recovered from a limestone quarry at Cornwell near Chipping Norton, Oxfordshire, in 1676. The fragment was sent to Robert Plot, Professor of Chemistry at the University of Oxford and first curator of the Ashmolean Museum, who published a description in his The Natural History of Oxford-shire (1677). He correctly identified the bone as the lower extremity of the femur of a large animal, and recognized that it was to

Vector embedding

In [25]:
from haystack import Document

question = "What is Mamenchisaurus?"

document = Document(content=question)
result = embedder.run(documents=[document])
embedded_document = result["documents"][0]
embedding = embedded_document.embedding

retrieval_result = retriever.run(query_embedding=embedding, top_k=1)
print(retrieval_result)
retrieved_documents = retrieval_result["documents"]
for doc in retrieved_documents:
    print(doc.content)

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.55it/s]

{'documents': [Document(id=c23833b2-e5bc-42c8-8a9d-7a5ffdf9b33a, content: 'Less well-preserved remains of the sauropodomorphs Jaklapallisaurus and Nambalia , along with the ea...', meta: {'split_id': 41.0, 'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'file_path': 'dinosaur-page.html'}, score: 0.7286020517349243, embedding: vector of size 1536)]}
Less well-preserved remains of the sauropodomorphs Jaklapallisaurus and Nambalia , along with the early saurischian Alwalkeria , are known from the Upper Maleri and Lower Maleri Formations of India. The Carnian-aged Chañares Formation of Argentina preserves primitive, dinosaur-like ornithodirans such as Lagosuchus and Lagerpeton in Argentina, making it another important site for understanding dinosaur evolution. These ornithodirans support the model of early dinosaurs as small, bipedal predators. Dinosaurs may have appeared as early as the Anisian epoch of the Triassic, approximately 243 million years ago, wh

Both vector embedding retriever and BM25 from Weaviate fail to fetch the right chunk.

The hypothesis for these experiments was that BM25 would perform better than vector embedding when the question is short, focussed on just a single term and is not semantically heavy. The expectation was that BM25 would have been able to search the unique terms like 'Mamenchisaurus' or 'Averostra' using BM25 as these terms are uncommon and inverted text based index should perform well here.

Two further questions:
1. Are there more nuances involved in successfully retrieving BM25 based chunks from Weaviate.  
    - Need to delve into Weaviate documentation - concepts, config options etc.
        - Explored briefly. The default settings seem reasonable, fine tuning ```k1``` and ```b``` parameters may help. But first I want to explore Elasticsearch. 
2. Will pure inverted text based solutions like Elasticsearch perform better than Weaviate BM25 (which is primarily a vector db)
    - Will explore Elasticsearch next

**Few more experiments**

In [26]:
bm25_retrieval_result = bm25retriever.run(query="What is Oryctodromeus?", top_k=1)
print(bm25_retrieval_result)
bm25_retrieved_documents = bm25_retrieval_result["documents"]
for doc in bm25_retrieved_documents:
    print(doc.content)

{'documents': [Document(id=a0ec45c3-c0cd-4604-a176-594456f11b36, content: 'Scholarly descriptions of what would now be recognized as dinosaur bones first appeared in the late ...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 23.0, 'file_path': 'dinosaur-page.html'}, score: 1.9453471899032593, embedding: vector of size 1536)]}
Scholarly descriptions of what would now be recognized as dinosaur bones first appeared in the late 17th century in England. Part of a bone, now known to have been the femur of a Megalosaurus , was recovered from a limestone quarry at Cornwell near Chipping Norton, Oxfordshire, in 1676. The fragment was sent to Robert Plot, Professor of Chemistry at the University of Oxford and first curator of the Ashmolean Museum, who published a description in his The Natural History of Oxford-shire (1677). He correctly identified the bone as the lower extremity of the femur of a large animal, and recognized that it was to

Wrong chunk. Elasticsearch BM25 was able to fetch correct chunk out of the box!

In [27]:
bm25_retrieval_result = bm25retriever.run(query="Who is Heilmann?", top_k=1)
print(bm25_retrieval_result)
bm25_retrieved_documents = bm25_retrieval_result["documents"]
for doc in bm25_retrieved_documents:
    print(doc.content)

{'documents': [Document(id=32e7c9bf-85c7-47c5-a9e4-e51d7cd1975f, content: 'The possibility that dinosaurs were the ancestors of birds was first suggested in 1868 by Thomas Hen...', meta: {'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'split_id': 99.0, 'file_path': 'dinosaur-page.html'}, score: 2.155226945877075, embedding: vector of size 1536)]}
The possibility that dinosaurs were the ancestors of birds was first suggested in 1868 by Thomas Henry Huxley. After the work of Gerhard Heilmann in the early 20th century, the theory of birds as dinosaur descendants was abandoned in favor of the idea of them being descendants of generalized thecodonts, with the key piece of evidence being the supposed lack of clavicles in dinosaurs. However, as later discoveries showed, clavicles (or a single fused wishbone, which derived from separate clavicles) were not actually absent; they had been found as early as 1924 in Oviraptor , but misidentified as an interclavicl

Correct chunk, both Elasticsearch and Weaviate BM25 predicted correctly. 'Heilmann' occurs twice in the document in the same chunk.

In [28]:
bm25_retrieval_result = bm25retriever.run(query="What is Compsognathus?", top_k=1)
print(bm25_retrieval_result)
bm25_retrieved_documents = bm25_retrieval_result["documents"]
for doc in bm25_retrieved_documents:
    print(doc.content)

{'documents': [Document(id=a0ec45c3-c0cd-4604-a176-594456f11b36, content: 'Scholarly descriptions of what would now be recognized as dinosaur bones first appeared in the late ...', meta: {'split_id': 23.0, 'source_id': 'cccb7d931975c65babedb9ad3867015939fba680621923f60517da5e5dea3c32', 'file_path': 'dinosaur-page.html'}, score: 1.9453471899032593, embedding: vector of size 1536)]}
Scholarly descriptions of what would now be recognized as dinosaur bones first appeared in the late 17th century in England. Part of a bone, now known to have been the femur of a Megalosaurus , was recovered from a limestone quarry at Cornwell near Chipping Norton, Oxfordshire, in 1676. The fragment was sent to Robert Plot, Professor of Chemistry at the University of Oxford and first curator of the Ashmolean Museum, who published a description in his The Natural History of Oxford-shire (1677). He correctly identified the bone as the lower extremity of the femur of a large animal, and recognized that it was to

Wrong chunk. Elasticsearch BM25 was able to fetch correct chunk out of the box!

Elasticsearch BM25 was atleast able to correctly identify chunks for following terms - Compsognathus, Oryctodromeus - which Weaviate BM25 was unable to. Elasticsearch BM25 seems better at least with these first few experiments. More to follow..